In [1]:
# %pip uninstall lion-core lionagi -y
# %pip install -e /Users/lion/lion-core/
# %pip install -e /Users/lion/lionagi/

In [2]:
import lionagi.operations as ops

In [3]:
from lionagi import __version__

print(f"Lionagi version {__version__}")

Lionagi version 0.3.8


In [4]:
from lion_service import iModel

gpt4o = iModel(
    provider="openai",
    api_key="OPENAI_API_KEY",
    model="gpt-4o",
    task="chat",
    temperature=0.1,
)

choices = [
    "cry",
    "call for help",
    "run towards",
    "fight",
    "flee",
    "scream",
    "hide",
]
instruction = "rank the following actions in order of preference"
guidance = "think carefully on the most effective choice, hint: certain obivious choices may not be the best"

In [5]:
from lion_core.session.branch import Branch

comedian = Branch()

response, branch = await ops.select(
    choices=choices,
    instruction=instruction,
    guidance=guidance,
    context="You are in a forest and you see a bear",
    reason=True,
    imodel=gpt4o,
    num_parse_retries=3,
    clear_messages=True,
    return_branch=True,
    branch_user=comedian,
)

In [6]:
branch.messages.to_df(columns=["ln_id", "sender", "recipient"])

,ln_id,sender,recipient
0,ln6e3961800f1927b-33ccbba0d-d-25-6ccdfdc77,ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977,ln06dbcf-a-1d0444-37169-19c65e863ffa262fd6
1,lnb96665-5532dea-8ef-c01e5d69-84aa6025e377,ln06dbcf-a-1d0444-37169-19c65e863ffa262fd6,ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977


In [7]:
from lionfuncs import as_readable
from IPython.display import display, Markdown

Markdown(as_readable(response, md=True))

```json
{
    "selected": [
        "flee"
    ],
    "reason": {
        "title": "Optimal Response to Bear Encounter",
        "content": "In a situation where you encounter a bear in the forest, the most effective action is to flee if it is safe to do so. This choice minimizes the risk of provoking the bear and increases the chances of escaping unharmed. Other actions like crying, calling for help, running towards, fighting, screaming, or hiding are less effective and could potentially escalate the situation.",
        "confidence_score": 0.923
    }
}
```

In [8]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-26 05:09:27.582294+00:00,Instruction,user,{'context': ['You are in a forest and you see ...,ln6e3961800f1927b-33ccbba0d-d-25-6ccdfdc77,ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977,ln06dbcf-a-1d0444-37169-19c65e863ffa262fd6,{'last_updated': {'recipient': 1729919369.9400...
1,2024-10-26 05:09:29.939683+00:00,AssistantResponse,assistant,"{'assistant_response': '```json { ""selecte...",lnb96665-5532dea-8ef-c01e5d69-84aa6025e377,ln06dbcf-a-1d0444-37169-19c65e863ffa262fd6,ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977,{'model_response': {'id': 'chatcmpl-AMTLENsLEO...


In [9]:
from lionfuncs import copy

config = copy(gpt4o.configs)
config.update({"model": gpt4o.model})

In [10]:
config

{'temperature': 0.1, 'model': 'gpt-4o'}

In [11]:
branch.user

'ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977'

In [12]:
branch.messages[0].formatted_dict

{'timestamp': datetime.datetime(2024, 10, 26, 5, 9, 27, 582294, tzinfo=datetime.timezone.utc),
 'ln_id': 'ln6e3961800f1927b-33ccbba0d-d-25-6ccdfdc77',
 'sender': 'ln37221cdd83b47-f7ab21faf6c4-f16-9-63ab977',
 'recipient': 'ln06dbcf-a-1d0444-37169-19c65e863ffa262fd6',
 'metadata': {'last_updated': {'recipient': 1729919369.940089}},
 'content': {'context': ['You are in a forest and you see a bear',
   {'respond_schema_info': {'$defs': {'ReasonModel': {'properties': {'title': {'anyOf': [{'type': 'string'},
          {'type': 'null'}],
         'default': None,
         'title': 'Title'},
        'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
         'default': None,
         'title': 'Content'},
        'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}],
         'default': None,
         'description': 'Provide an objective numeric confidence score between 0 and 1 (with 3 decimal places) indicating how likely you successfully achieved the task according t

In [13]:
res2, branch = await ops.brainstorm(
    num_steps=3,
    instruction=f"{instruction}\nChoices:{choices}",
    guidance=guidance,
    context="You are in a forest and you see a bear",
    imodel=gpt4o,
    num_parse_retries=3,
    clear_messages=True,
    reason=True,
    return_branch=True,
)

In [14]:
Markdown(as_readable(res2, md=True))

```json
{
    "topic": "",
    "ideas": [],
    "reason": {
        "title": "Ranking Actions When Encountering a Bear",
        "content": "When encountering a bear in the forest, the primary goal is to ensure personal safety. The most effective actions are those that minimize the risk of provoking the bear or putting oneself in danger. Based on general wildlife safety guidelines, the recommended order of actions is as follows: 1) Hide - This action minimizes visibility and potential threat to the bear. 2) Flee - If the bear has not noticed you, quietly moving away can be a safe option. 3) Call for help - If you have a phone or device, calling for help can bring assistance. Actions like 'cry', 'scream', 'run towards', and 'fight' are less preferred as they can provoke the bear or increase danger. Therefore, the ranking is: hide, flee, call for help, cry, scream, run towards, fight.",
        "confidence_score": 0.95
    }
}
```

In [15]:
branch.messages[1]

AssistantResponse(ln_id='ln6e31ce37a7-c6aaba05-b-bbac3190-d915051c1', timestamp=2024-10-26 05:09:32.683534, content={'assistant_response': '```json\n{\n    "reason": {\n        "title": "Ranking Actions When Encou..., metadata={'model_response': {'id': 'chatcmpl-AMTLGmloC0F0n7EZsK6fkQfnb7nh1', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': '```json\n{\n    "reason": {\n        "title": "Ranking Actions When Encountering a Bear",\n        "content": "When encountering a bear in the forest, the primary goal is to ensure personal safety. The most effective actions are those that minimize the risk of provoking the bear or putting oneself in danger. Based on general wildlife safety guidelines, the recommended order of actions is as follows: 1) Hide - This action minimizes visibility and potential threat to the bear. 2) Flee - If the bear has not noticed you, quietly moving away can be a safe option. 3) Call for help - If you have a phone or device, calling for help

In [16]:
from enum import Enum


class MessageField(str, Enum):

    TIMESTAMP = "timestamp"
    LION_CLASS = "lion_class"
    ROLE = "role"
    CONTENT = "content"
    LN_ID = "ln_id"
    SENDER = "sender"
    RECIPIENT = "recipient"
    METADATA = "metadata"

    @classmethod
    def fields(cls):
        return [i.value for i in cls.__members__.values()]


MessageField.fields()

['timestamp',
 'lion_class',
 'role',
 'content',
 'ln_id',
 'sender',
 'recipient',
 'metadata']

In [17]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-26 05:09:30.025494+00:00,Instruction,user,{'context': [{'operation': 'Please follow prom...,ln487d73b-fba76e8be8-6bb0-bfbfad-8c3c40e07,user,ln55125163-4062a1c74-a0679-536-ee3ff38b159,{'last_updated': {'recipient': 1729919372.6837...
1,2024-10-26 05:09:32.683534+00:00,AssistantResponse,assistant,"{'assistant_response': '```json { ""reason""...",ln6e31ce37a7-c6aaba05-b-bbac3190-d915051c1,ln55125163-4062a1c74-a0679-536-ee3ff38b159,user,{'model_response': {'id': 'chatcmpl-AMTLGmloC0...


In [18]:
res2

BrainstormModelResponse(topic='', ideas=[], reason=ReasonModel(title='Ranking Actions When Encountering a Bear', content="When encountering a bear in the forest, the primary goal is to ensure personal safety. The most effective actions are those that minimize the risk of provoking the bear or putting oneself in danger. Based on general wildlife safety guidelines, the recommended order of actions is as follows: 1) Hide - This action minimizes visibility and potential threat to the bear. 2) Flee - If the bear has not noticed you, quietly moving away can be a safe option. 3) Call for help - If you have a phone or device, calling for help can bring assistance. Actions like 'cry', 'scream', 'run towards', and 'fight' are less preferred as they can provoke the bear or increase danger. Therefore, the ranking is: hide, flee, call for help, cry, scream, run towards, fight.", confidence_score=0.95))

In [19]:
from lionfuncs import as_readable
from IPython.display import Markdown

import lionagi.operations as ops

res2, session = await ops.rank(
    choices=choices,
    num_scorers=10,
    instruction=instruction,
    guidance=guidance,
    context="You are in a forest and you see a bear",
    imodel=gpt4o,
    num_parse_retries=0,
    clear_messages=True,
    reason=True,
    return_session=True,
)

Markdown(as_readable(res2, md=True))

```json
{
    "item": "flee",
    "scores": [
        8.0,
        8.0,
        9.0,
        9.0,
        8.0,
        9.0,
        9.0,
        9.0,
        9.0,
        9.0
    ],
    "average": 8.7
}

{
    "item": "hide",
    "scores": [
        8.0,
        9.0,
        8.0,
        8.0,
        8.0,
        8.0,
        8.0,
        8.0,
        8.0
    ],
    "average": 8.11111111111111
}

{
    "item": "call for help",
    "scores": [
        8.0,
        8.0,
        8.0,
        7.0,
        8.0,
        8.0,
        8.0,
        8.0,
        8.0,
        8.0
    ],
    "average": 7.9
}

{
    "item": "scream",
    "scores": [
        3.0,
        4.0,
        3.0,
        4.0,
        4.0,
        4.0,
        3.0,
        3.0,
        4.0,
        4.0
    ],
    "average": 3.6
}

{
    "item": "fight",
    "scores": [
        3.0,
        3.0,
        2.0,
        3.0,
        2.0,
        3.0,
        3.0,
        3.0,
        3.0,
        3.0
    ],
    "average": 2.8
}

{
    "item": "cry",
    "scores": [
        3.0,
        2.0,
        3.0,
        2.0,
        2.0,
        3.0,
        2.0,
        2.0,
        2.0,
        3.0
    ],
    "average": 2.4
}

{
    "item": "run towards",
    "scores": [
        2.0,
        2.0,
        2.0,
        2.0,
        2.0,
        2.0,
        2.0,
        2.0,
        2.0,
        2.0
    ],
    "average": 2.0
}
```

In [20]:
df = session.to_df()

In [21]:
len(df)

140

In [22]:
df.head()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-26 05:09:32.742502+00:00,Instruction,user,{'context': [{'operation': 'rank the following...,lne2661c-f528-092-fcce-edecb9a92f4a484bff0,ln2e575aca7b08b551-93dd-3-5ed6b6-22a321d68,ln5803cf4cdfed-746f392-cc4ce0c1-23-2f41ca2,{'last_updated': {'recipient': 1729919376.9373...
1,2024-10-26 05:09:36.937045+00:00,AssistantResponse,assistant,"{'assistant_response': '```json { ""score"":...",ln463e2b1-fa5-826bc721e9-5-8398a9249713e68,ln5803cf4cdfed-746f392-cc4ce0c1-23-2f41ca2,user,{'model_response': {'id': 'chatcmpl-AMTLKVZOFW...
2,2024-10-26 05:09:32.745976+00:00,Instruction,user,{'context': [{'operation': 'rank the following...,lnd101d9d9ca6636c1f2-eb-b99-e23e-6a655f192,ln2e575aca7b08b551-93dd-3-5ed6b6-22a321d68,ln5dbbf9-08171-af-3f4-575e38d089cb1cafd0f9,{'last_updated': {'recipient': 1729919375.6597...
3,2024-10-26 05:09:35.659616+00:00,AssistantResponse,assistant,"{'assistant_response': '```json { ""score"": 8...",ln934c0db25932151-f412-5-b1-a9fa82356980ae,ln5dbbf9-08171-af-3f4-575e38d089cb1cafd0f9,user,{'model_response': {'id': 'chatcmpl-AMTLK4ssZS...
4,2024-10-26 05:09:32.749404+00:00,Instruction,user,{'context': [{'operation': 'rank the following...,lne9e349e7a-f6ec-d02011-1d89-ff8dd8c9b4bc2,ln2e575aca7b08b551-93dd-3-5ed6b6-22a321d68,ln8b02bf9ad-1-f82c2172c6b051-65-6da06d96e0,{'last_updated': {'recipient': 1729919376.1727...
